In [ ]:
import cryptocompare
import time
import datetime
import pandas as pd
import numpy as np
import os
import shutil

In [ ]:
CCList = ['BTC', 'XRP', 'ETH', 'TEMCO', 'BORA', 'SAND', 'WEMIX', 'DOGE', 'LUNA', 'SOL', 'KLAY']  
# 코빗에는 TEMCO와 BORA가 없음
EXList = ['Coinone', 'Bithumb', 'Korbit']

# 빗썸 거래량 상위 5위와 이름 들어본 것들
# 22. 2. 6. 기준 빗썸 24시간 거래금액 1억 이상


Cryptocompare 에서 제공하는 daily API에는 allData 라는 parameter로 모든 데이터를 긁어올 수 있다.

그래서 매번 모든 데이터를 받아서 단순 저장한다

In [ ]:
#history, daily
for EX in EXList:
    for CC in CCList:
        filename = "data/{}_{}_KRW_day_gmt.csv22".format(EX, CC)
        print("collecting {}...".format(filename))
        allHist = cryptocompare.get_historical_price_day_all(CC, 'KRW', EX)
        if allHist is None:
            print("cryptocompare API failed!")
            time.sleep(5)
            # 1번만 더 재시도해본다
            allHist = cryptocompare.get_historical_price_day_all(CC, 'KRW', EX)
            if allHist is None:
                print("cryptocompare API failed again!!!")
                continue
        df = pd.DataFrame(allHist, columns = [
                    'time', 'open', 'high', 'low', 'close', 
                    'volumefrom', 'volumeto', 
                    'conversionType', 'conversionSymbol'])
        df.rename(inplace=True, columns={
            "volumefrom": "volume",
            "volumeto": "value"
        })
        df = df.set_index('time')
        df = df.sort_index()

        # 오늘 날짜 일봉은 종가를 믿을 수 없으므로 제외
        # 여기서 '오늘'은 GMT 기준으로 오늘임을 잊지 말것
        df.drop(df.index[-1:], inplace=True)

        df.index = pd.to_datetime(df.index, unit='s')
        df.index = df.index.tz_localize('GMT')
        df.to_csv(filename)
        time.sleep(2)


hour 데이터는 기존에 있는 데이터를 불러온뒤 마지막 시간대 이후 데이터만 cryptocompare에 요청하여 긁어오고 추가한다.

In [ ]:
# history, hourly

for EX in EXList:
    for CC in CCList:
        filename = "data/{}_{}_KRW_hour_gmt.csv".format(EX, CC)
        print("collecting {}...".format(filename))
        if os.path.isfile(filename):
            dt_parser = lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S%z")
            hist_hour = pd.read_csv(
                filename, 
                index_col = 0, 
                parse_dates = True, 
                date_parser = dt_parser)
            fromTs = int(np.max(hist_hour.index).timestamp())
            print("past data file existed, using fromTs:{}". format(fromTs))
            shutil.copy(filename, filename+".bak")
        else:
            hist_hour = None
            fromTs = 0

        toTs = time.time() - 7200  # 2시간 전까지만의 시간봉을 얻는다

        apiHist = cryptocompare.get_historical_price_hour_from(CC, 'KRW', EX, toTs, fromTs, delay=1)
        if apiHist is None:
            print("cryptocompare API failed!!!")
            time.sleep(5)
            # 1번만 더 재시도해본다
            apiHist = cryptocompare.get_historical_price_hour_from(CC, 'KRW', EX, toTs, fromTs, delay=2)
            if apiHist is None:
                print("cryptocompare API failed again!!!")
                continue

        hist_hour_api = pd.DataFrame(apiHist, columns = [
            'time', 'open', 'high', 'low', 'close', 
            'volumefrom', 'volumeto', 
            'conversionType', 'conversionSymbol'])
        hist_hour_api.rename(inplace=True, columns={
            "volumefrom": "volume",
            "volumeto": "value"
        })
        hist_hour_api = hist_hour_api.set_index('time')
        hist_hour_api = hist_hour_api.sort_index()
        hist_hour_api.index = pd.to_datetime(hist_hour_api.index, unit='s')
        hist_hour_api.index = hist_hour_api.index.tz_localize('GMT')

        hist_hour = pd.concat([hist_hour, hist_hour_api])
        hist_hour.to_csv(filename)
        time.sleep(1)



다음은 cryptocompare 패키지에서 추가한 메소드

```python
def get_historical_price_day_all(coin: str, currency: str = CURRENCY,
                                 exchange: str = 'CCCAGG') -> Optional[Dict]:
    """
    Get historical price (day, all).

    :param coin: symbolic name of the coin (e.g. BTC)
    :param currency: short hand description of the currency (e.g. EUR)
    :param exchange: exchange to use (default: 'CCCAGG')
    :returns: dict of coin and currency price pairs
    """
    response = _query_cryptocompare(
        _URL_HIST_PRICE_DAY.format(coin, _format_parameter(currency), 1, exchange, time.time()) + "&allData=true")
    if response:
        return response['Data']['Data']
    return None


def get_historical_price_hour_from(coin: str, currency: str = CURRENCY,
                                 exchange: str = 'CCCAGG', toTs: Timestamp = time.time(),
                                 fromTs: Timestamp = 0, delay: float = 0.2) -> Optional[Dict]:
    """
    Get historical price (day).

    :param coin: symbolic name of the coin (e.g. BTC)
    :param currency: short hand description of the currency (e.g. EUR)
    :param exchange: exchange to use (default: 'CCCAGG')
    :param toTs: return data before this timestamp. (Unix epoch time or datetime object)
    :param fromTs: return data after this timestamp. (Unix epoch time or datetime object)
    :param delay: time delay for API rate limit (default: 300 calls / 1 minute)
    :returns: dict of coin and currency price pairs
    """
    allHist = []

    while fromTs <= toTs:
        p = get_historical_price_hour(coin, currency, LIMIT, exchange, toTs)
        if p is None:
            return None

        validHist = [elem for elem in p if elem['time'] >= fromTs and elem['open'] != 0 and elem['close'] != 0]
        allHist = validHist + allHist

        if len(validHist) < len(p):
            break
        toTs = (min(p, key = lambda x:x['time']))['time'] - 1
        time.sleep(delay)

    return allHist
```
